In [1]:
import monte_carlo as m

df = m.simulate_hsv_sample(1000, 0.6, 0.8, 100, 0.5)
df

,income,net_income,working,household_size
0,98206.291409,5479.514099,3,4
1,13765.295178,5261.975811,2,3
2,10794.103947,2174.276219,3,4
3,13655.472819,6742.728250,2,5
4,43814.754712,6269.253164,3,4
...,...,...,...,...
995,19411.937318,3637.526295,3,4
996,31574.503998,8330.975916,1,2
997,33022.265223,5502.870544,4,5
998,7617.717539,3411.129442,2,4


## OLS

In [2]:
model = m.generate_ols_model(df)

# Note that estimated tau is 1 - (coefficient on income).
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.554
Model:                            OLS   Adj. R-squared:                  0.553
Method:                 Least Squares   F-statistic:                     619.0
Date:                Fri, 21 Mar 2025   Prob (F-statistic):          1.69e-175
Time:                        12:37:57   Log-Likelihood:                -753.53
No. Observations:                1000   AIC:                             1513.
Df Residuals:                     997   BIC:                             1528.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
==================================================================================
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
const              4.5110      0.205     21.997      0.000       4.109       4.913
income             0.4050      0.020     19.950      0.000       0.365       0.445
household_size     0.8397      0.029     28.666      0.000       0.782       0.897
==============================================================================
Omnibus:                        0.564   Durbin-Watson:                   1.931
Prob(Omnibus):                  0.754   Jarque-Bera (JB):                0.546
Skew:                           0.057   Prob(JB):                        0.761
Kurtosis:                       2.999   Cond. No.                         128.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [3]:
# Sigma estimate
sigma_hat = m.get_sigma_hat(model)
sigma_hat

0.5140637700630478

## Maximum Likelihood

In [4]:
initial_params = [0.6, 0.8, 0.5]
model = m.generate_mle_model(df, initial_params)
model.x

array([-0.33091997,  0.82508864,  1.67489065])